In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd

In [ ]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)
writer.writerow(('name','url'))
urlList = []





In [ ]:
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                 "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  



In [ ]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [ ]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))



In [ ]:
pages = driver.find_element_by_class_name("page-total").text




In [ ]:
#pages = 1

In [ ]:
for page in range(1,int(pages)+1):
    if (page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "current")))
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sf-item-list-data")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    jd = jsonData["data"]
    
    itemLen = len(jd)
    for item in range(itemLen):    
    #for i in range(6):        
        itemStatus = jd[item]["status"]
        
        if(itemStatus == "done"):
                   
            num = jd[item]["id"]
            url = "#pai-item-"+str(num)
            urlList.append(url)
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
           
            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))

            handles = driver.window_handles
            time.sleep(1) # have to wait sometime
            driver.switch_to.window(handles[1])
            #
            url2 = str(driver.current_url)+"#J_Confirmation"
            driver.get(url2)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "c-content")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            # getName
            name = info[4:index]
            print("Name:"+name+": Page :"+str(page)+"   Item :"+str(item))
            print("url:"+driver.current_url)
            # add to file
            writer.writerow((name,driver.current_url))
            driver.close()
            driver.switch_to.window(handles[0])
        else:
            print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



In [ ]:
driver.quit()

In [ ]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')